In [ ]:
# Installation of required libraries
!pip install statsmodels
!pip install dcor
!pip install tigramite
!pip install sklearn

In [ ]:
# Import all required libraries
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller, kpss
from sklearn.metrics.pairwise import rbf_kernel
import matplotlib.pyplot as plt
from tigramite import data_processing as pp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.cmiknn import CMIknn

In [ ]:
# Data Loading Functions
def load_dataset(df, time_col=None, feature_cols=None):
    """
    Load and process time series data

    Parameters
    ----------
    df : pandas.DataFrame
        Input dataframe
    time_col : str or None
        Name of the time column. If None, creates sequential time index
    feature_cols : list
        List of feature columns to use
    """
    # Handle time column
    if time_col is None:
        # Create sequential time index
        df = df.copy()  # Create a copy to avoid modifying original
        df['time'] = np.arange(len(df))
        time_col = 'time'
        print("Created sequential time index")
    elif time_col not in df.columns:
        print(f"Warning: Time column '{time_col}' not found. Creating sequential time index.")
        df = df.copy()
        df['time'] = np.arange(len(df))
        time_col = 'time'

    # Select features
    if feature_cols is None:
        feature_cols = df.select_dtypes(include=[np.number]).columns
        feature_cols = [col for col in feature_cols if col != time_col]
    else:
        missing_cols = [col for col in feature_cols if col not in df.columns]
        if missing_cols:
            raise ValueError(f"Feature columns {missing_cols} not found")

    return df[[time_col] + feature_cols]

In [ ]:
# Decomposition Functions
def find_optimal_stl_period(series, max_period=50, variance_threshold=0.1):
    """Find optimal period for STL decomposition"""
    best_period = None
    best_seasonal_variance = 0

    for period in range(2, max_period + 1):
        stl = STL(series, period=period, seasonal=13)
        result = stl.fit()
        seasonal_variance = np.var(result.seasonal)

        if seasonal_variance > best_seasonal_variance and seasonal_variance > variance_threshold:
            best_period = period
            best_seasonal_variance = seasonal_variance

    return best_period

def decompose_series(series, period):
    """Decompose single time series into components"""
    if period is None:
        return series, np.zeros(len(series)), series - series.mean()

    seasonal = 15
    trend = max(int(period) + 1, 35)
    if trend % 2 == 0:
        trend += 1

    stl = STL(series, seasonal=seasonal, trend=trend, period=int(period))
    result = stl.fit()
    return result.trend, result.seasonal, result.resid

def decompose_all(df, time_col='time', max_period=50, variance_threshold=0.1):
    """Decompose all time series in dataframe"""
    components = {}
    optimal_periods = {}
    feature_cols = [col for col in df.columns if col != time_col]

    # Find optimal periods
    for col in feature_cols:
        optimal_period = find_optimal_stl_period(df[col], max_period, variance_threshold)
        optimal_periods[col] = optimal_period
        print(f"Using period {optimal_period} for {col}")

    # Decompose each series
    for col in feature_cols:
        trend, seasonal, resid = decompose_series(df[col], optimal_periods[col])
        components[f'{col}_trend'] = trend
        components[f'{col}_seasonal'] = seasonal
        components[f'{col}_residual'] = resid

    return pd.DataFrame(components), optimal_periods

In [ ]:
# Cell 5: Analysis Functions
def check_stationarity(series, series_name):
    """Check stationarity of a time series"""
    adf_result = adfuller(series)
    kpss_result = kpss(series, regression='c')
    is_stationary = adf_result[1] < 0.05 and kpss_result[1] > 0.05

    if not is_stationary:
        print(f"t --> {series_name.replace('_trend', '')}")
    return is_stationary

def run_pcmci_analysis(components_df, max_lag=2):
    """Run PCMCI+ analysis on residual components"""
    residual_cols = [col for col in components_df.columns if 'residual' in col]
    data = components_df[residual_cols].values

    dataframe = pp.DataFrame(data, var_names=residual_cols)
    cmi = CMIknn(significance='shuffle_test', knn=0.2, shuffle_neighbors=5,
                 transform='ranks', workers=-1)
    pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cmi, verbosity=1)

    results = pcmci.run_pcmciplus(tau_min=0, tau_max=max_lag, pc_alpha=0.05)

    # Print significant links
    pcmci.print_significant_links(p_matrix=results['p_matrix'],
                                val_matrix=results['val_matrix'],
                                alpha_level=0.05)
    return results

In [ ]:
# Analysis Functions
def hsic_test_with_time(seasonal_component, time, gamma=0.1, num_permutations=500,
                       variance_threshold=0.01):
    """Perform HSIC test for time dependency"""
    if np.var(seasonal_component) < variance_threshold:
        print(f"Skipping HSIC test due to low variance: {np.var(seasonal_component):.4f}")
        return None, None

    K_time = rbf_kernel(time, gamma=gamma)
    L_seasonal = rbf_kernel(seasonal_component.values.reshape(-1, 1), gamma=gamma)
    n = K_time.shape[0]
    H = np.eye(n) - np.ones((n, n)) / n
    HSIC = (1 / (n - 1) ** 2) * np.trace(K_time @ H @ L_seasonal @ H)

    hsic_values = []
    for _ in range(num_permutations):
        perm = np.random.permutation(n)
        L_perm = L_seasonal[perm][:, perm]
        hsic_values.append((1 / (n - 1) ** 2) * np.trace(K_time @ H @ L_perm @ H))

    p_value = np.mean([1 if hsic_perm > HSIC else 0 for hsic_perm in hsic_values])
    return HSIC, p_value

def run_pcmci_analysis(components_df, max_lag=2):
    """Run PCMCI+ analysis on residual components"""
    residual_cols = [col for col in components_df.columns if 'residual' in col]
    data = components_df[residual_cols].values

    dataframe = pp.DataFrame(data, var_names=residual_cols)
    cmi = CMIknn(significance='shuffle_test', knn=0.2, shuffle_neighbors=5,
                 transform='ranks', workers=-1)
    pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cmi, verbosity=1)

    results = pcmci.run_pcmciplus(tau_min=0, tau_max=max_lag, pc_alpha=0.05)
    pcmci.print_significant_links(p_matrix=results['p_matrix'],
                                val_matrix=results['val_matrix'],
                                alpha_level=0.05)
    return results

In [ ]:
# Visualization Functions
def plot_decomposition(data, components_df, time_col='time'):
    """Plot original data and decomposed components"""
    feature_cols = [col for col in data.columns if col != time_col]

    for column in feature_cols:
        fig, axes = plt.subplots(4, 1, figsize=(10, 8), sharex=True)
        fig.suptitle(f"Decomposition of {column}")

        axes[0].plot(data[time_col], data[column], label='Raw Data', color='blue')
        axes[0].set_ylabel('Raw Data')
        axes[0].legend()

        axes[1].plot(data[time_col], components_df[f'{column}_trend'],
                    label='Trend', color='green')
        axes[1].set_ylabel('Trend')
        axes[1].legend()

        axes[2].plot(data[time_col], components_df[f'{column}_seasonal'],
                    label='Seasonal', color='orange')
        axes[2].set_ylabel('Seasonal')
        axes[2].legend()

        axes[3].plot(data[time_col], components_df[f'{column}_residual'],
                    label='Residual', color='red')
        axes[3].set_ylabel('Residual')
        axes[3].legend()

        plt.xlabel('Time')
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

In [ ]:
# Main Execution with proper file handling
def main():
    # Upload file
    print("Please upload your dataset (CSV or Excel file)...")
    uploaded = files.upload()

    # Get the filename and load the file
    filename = list(uploaded.keys())[0]

    # Create a proper file-like object from the bytes
    import io
    if filename.endswith('.csv'):
        df_temp = pd.read_csv(io.StringIO(uploaded[filename].decode('utf-8')))
    else:
        df_temp = pd.read_excel(io.BytesIO(uploaded[filename]))

    print("\nFirst few rows of your data:")
    print(df_temp.head())
    print("\nAvailable columns:", df_temp.columns.tolist())
    print("Dataset shape:", df_temp.shape)

    # Get parameters from user
    has_time = input("\nDoes your dataset have a time column? (y/n): ").lower() == 'y'
    if has_time:
        time_col = input("Enter the name of time column: ")
    else:
        time_col = None
        print("Will create sequential time index")

    max_lag = int(input("Enter maximum lag period (default 2): ") or 2)

    # Feature selection
    use_all = input("\nUse all numeric features? (y/n): ").lower() == 'y'
    if use_all:
        feature_cols = None
    else:
        features_input = input("Enter feature column names (comma-separated): ")
        feature_cols = [f.strip() for f in features_input.split(',')]

    # Load and process data
    print("\nProcessing data...")
    df = load_dataset(df_temp, time_col, feature_cols)

    # Rest of the processing
    print("\nDecomposing time series...")
    components_df, optimal_periods = decompose_all(df, 'time')

    print("\nGenerating plots...")
    plot_decomposition(df, components_df, 'time')

    print("\nChecking stationarity of trend components:")
    for var in [col.replace('_trend', '') for col in components_df.columns if 'trend' in col]:
        check_stationarity(components_df[f'{var}_trend'], f'{var}_trend')

    print("\nRunning PCMCI+ analysis...")
    pcmci_results = run_pcmci_analysis(components_df, max_lag)

    return df, components_df, pcmci_results, optimal_periods

if __name__ == "__main__":
    df, components_df, results, optimal_periods = main()